In [1]:
import sys 
sys.path.append("..")
import matplotlib.pyplot as plt 
from utils.utils_pcl import *
from utils.utils_pointcloud import *
import cv2 
import rospy 
from get_rostopic import RealsenseD435i
print("Done.")

Done.


In [2]:
def do_ransac_plane_normal_segmentation(point_cloud, input_max_distance):
    segmenter = point_cloud.make_segmenter_normals(ksearch=50)
    segmenter.set_optimize_coefficients(True)
    segmenter.set_model_type(pcl.SACMODEL_NORMAL_PLANE)  #pcl_sac_model_plane
    segmenter.set_normal_distance_weight(0.1)
    segmenter.set_method_type(pcl.SAC_RANSAC) #pcl_sac_ransac
    segmenter.set_max_iterations(100)
    segmenter.set_distance_threshold(input_max_distance) #0.03)  #max_distance
    indices, coefficients = segmenter.segment()

    print('Model coefficients: ' + str(coefficients[0]) + ' ' + str(
        coefficients[1]) + ' ' + str(coefficients[2]) + ' ' + str(coefficients[3]))

    print('Model inliers: ' + str(len(indices)))
    for i in range(0, 5):#range(0, len(indices)):
        print(str(indices[i]) + ', x: ' + str(point_cloud[indices[i]][0]) + ', y : ' +
              str(point_cloud[indices[i]][1]) + ', z : ' + str(point_cloud[indices[i]][2]))

    inliers = point_cloud.extract(indices, negative=False)
    outliers = point_cloud.extract(indices, negative=True)

    return indices, inliers, outliers

### Setup

In [4]:
rospy.init_node('Realsense')
mode     = "pointcloud"
camera   = RealsenseD435i(mode=mode)
name = "1025"


### RANSAC

In [5]:
point = ros_to_pcl(camera.point_cloud)
print(point)
cloud_voxel = do_voxel_grid_downssampling(point, 0.01)
cloud_pass_z = do_passthrough(cloud_voxel, 'x', axis_min=0.01, axis_max=1.5)

outliner_object = do_ransac_plane_segmentation(cloud_pass_z,pcl.SACMODEL_PLANE,pcl.SAC_RANSAC,0.01)
outliner_object = outliner_object.to_array()
print(outliner_object.shape)

<PointCloud of 115002 points>
(18150, 4)
